In [2]:
from flask import Flask,render_template,request, redirect, make_response, url_for, session,Response
import pickle
import nltk
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from rake_nltk import Rake


app = Flask(__name__)   

###     parameters     ###

user_msg = []
bot_msg = []
user_symp = []


###     function sections starts    ###
def random_resp(df):
    text = df['0'].sample(n=1)
    text = np.array(text)
    return (''.join(text))

def lemma (text):
    WNlemma = nltk.WordNetLemmatizer()
    return (WNlemma.lemmatize(text))
def stem(text):
    porter = nltk.PorterStemmer()
    return (porter.stem(text))
def list2string(text):   
    return( ''.join(text))

class RakeImpl:
    def __init__(self, text):
        self.text = text
        self.rake = Rake()
    def getKeywords(self):
        self.rake.extract_keywords_from_text(self.text)
        return self.rake.get_ranked_phrases()

def preprocess(audio_text):
    preprocess_1 = stem(audio_text)
    preprocess_2 = lemma(preprocess_1)
    preprocess_3 = list2string(preprocess_2)
    rakeImpl = RakeImpl(preprocess_3)
    symp_extr = rakeImpl.getKeywords()[:20]
    symp = list2string(symp_extr) 
    return (predict(symp))

def predict (symp):
    user_symp.append(symp)
    y_pred = CPS_Model.predict_proba(user_symp)
    pred = CPS_Model.predict(user_symp)
    return (pred, y_pred.max())

def core (user_input, discard):
    dieases, dieases_prob = preprocess(user_input)
    while dieases_prob < 0.90 :
        if discard == False:
            return add_bot_msg(random_resp(symp_error))
        else:
            return (dieases, dieases_prob)
    return (dieases, dieases_prob)


###      routing section starts     ###
@app.route('/',methods=['POST','GET'])    
def home():
    if request.method=='POST':
        user_name= request.form['user_name']
        user_age= request.form['user_age']
        resp=make_response(redirect('/messager'))
        resp.set_cookie('User_name',user_name) 
        resp.set_cookie('user_age',user_age)
        print(user_name,user_age)
        return resp
    else:
        return render_template('index.html');  

@app.route('/feedback')    
def feedback():
    
    return render_template('feedback.html');  

@app.route('/messager',methods =["GET","POST"])    
def messager():
    if request.method == "POST":
        current_msg = request.form['msg']
        add_user_msg(current_msg)
        print(user_msg)

        length = max(len(user_msg),len(bot_msg))        
        voice_msg= bot_msg[len(bot_msg)-1]

        return render_template('messager.html',bot_msg = bot_msg, user_msg=user_msg, length= length, voice_msg= voice_msg)
    
    else:
        return render_template('messager.html',bot_msg = bot_msg, user_msg=user_msg, length= len(bot_msg)) 

###    routing section ends    ###
if __name__ =='__main__':  
    
###  importing csv files   ###
    greet_ques = pd.read_csv('response//greet_ques_db.csv')
    intro = pd.read_csv('response//intro_db.csv')
    symp_greet = pd.read_csv('response//symp_greet_db.csv')
    symp = pd.read_csv('response//symp_db.csv')
    symp_error = pd.read_csv('response//symp_error_db.csv')

    des = pd.read_csv('csv/symptom_Description.csv')
    prec = pd.read_csv('csv/symptom_precaution.csv')
    sev = pd.read_csv('csv/Symptom-severity.csv')

###   importing machine learning model  ###
    with open("CPS.pkl", 'rb') as file:  
        CPS_Model = pickle.load(file)
    
    
    def add_user_msg(msg):
        user_msg.append(msg)

    def add_bot_msg(msg):
        bot_msg.append(msg)

###     response generation    ###

    add_bot_msg(random_resp(intro))
        


###     flask functions    ###
    app.run(debug = True)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1